In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("./slide10_dataset.csv")

In [3]:
data

,Temperature,Outlook,Humidity,Windy,Played
0,Mild,Sunny,80,0,1
1,Hot,Sunny,75,1,0
2,Hot,Overcast,77,0,1
3,Cool,Rain,70,0,1
4,Cool,Overcast,72,1,1
5,Mild,Sunny,77,0,0
6,Cool,Sunny,70,0,1
7,Mild,Rain,69,0,1
8,Mild,Sunny,65,1,1
9,Mild,Overcast,77,1,1


'Sunny'